
#Майнор ИАД. Домашнее задание 3. .

В этом задании вы напишете и обучите свой собственный YOLO детектор. Нужно будет разобраться со статьей: понять какого формата должна быть обучающая пара (x, y), как перевести лосс из математической формулы в питоновский код - ну и конечно понять и реализовать саму архитектуру модели.

Выборка на котрой мы будем обучать модель состоит из разнообразных фотографий яблок, бананов и апельсинов. Данные скачиваем [отсюда](https://drive.google.com/file/d/1d8GSfZoWbraWCSUhX78yl4CnMFYE-5n3/view?usp=sharing).

Баллы за ДЗ распределены следующим образом: 
- Выборка для YoloV1 - 2 балла
- YOLO модель - 2 балла
- YOLO Loss - 3 балла
- Вспомогательные функции - 2 балла
- Обучение и расчет метрик - 2 балла

Для построения и обучения можно использовать как pytorch, так и pytorch-lightning.

Да-да, баллов в сумме получается 11

In [6]:
# Данная библиотека понадобится нам, чтобы обработать разметку
! pip install xmltodict pytorch-lightning

Скачаем данные

In [3]:
!wget --quiet --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://drive.google.com/uc?export=download&id=1d8GSfZoWbraWCSUhX78yl4CnMFYE-5n3' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1d8GSfZoWbraWCSUhX78yl4CnMFYE-5n3" -O data.zip && rm -rf /tmp/cookies.txt
!unzip -q data.zip
!rm data.zip
!ls -l

total 64
drwxr-xr-x  4 fedor  staff    128 Dec  1 12:27 __MACOSX
drwx------  5 fedor  staff    160 Nov 26 11:32 data
-rw-r--r--  1 fedor  staff  31386 Dec  1 12:27 hw3.ipynb.ipynb


Посмотрим как выглядит один из файлов разметки

In [4]:
!cat data/train/apple_3.xml

<annotation>
	<folder>train</folder>
	<filename>apple_3.jpg</filename>
	<path>C:\tensorflow1\models\research\object_detection\images\train\apple_3.jpg</path>
	<source>
		<database>Unknown</database>
	</source>
	<size>
		<width>1000</width>
		<height>708</height>
		<depth>3</depth>
	</size>
	<segmented>0</segmented>
	<object>
		<name>apple</name>
		<pose>Unspecified</pose>
		<truncated>1</truncated>
		<difficult>0</difficult>
		<bndbox>
			<xmin>584</xmin>
			<ymin>438</ymin>
			<xmax>867</xmax>
			<ymax>708</ymax>
		</bndbox>
	</object>
	<object>
		<name>apple</name>
		<pose>Unspecified</pose>
		<truncated>0</truncated>
		<difficult>0</difficult>
		<bndbox>
			<xmin>492</xmin>
			<ymin>141</ymin>
			<xmax>740</xmax>
			<ymax>394</ymax>
		</bndbox>
	</object>
	<object>
		<name>apple</name>
		<pose>Unspecified</pose>
		<truncated>0</truncated>
		<difficult>0</difficult>
		<bndbox>
			<xmin>176</xmin>
			<ymin>199</ymin>
			<xmax>490</xmax>
			<ymax>466</ymax>
		</bndbox>
	</object>
	<obj

## Релизуйте выборку для YoloV1 - 2 балла

In [1]:
import os
import cv2
import json
import glob
import tqdm
import xmltodict

from IPython.core.display import struct

from typing import List

import pandas as pd
import numpy as np

import torch
import torchvision
from torch import nn
from torch.nn import functional as F
from torch.utils.data import Dataset, DataLoader

import pytorch_lightning as pl

import albumentations as A
import albumentations.pytorch

from PIL import Image

import matplotlib.pyplot as plt

from sklearn.metrics import auc
# Добавьте необходимые вам библиотеки, если их не окажется в списке выше

import xml.etree.ElementTree as ET #for xml parsing
import warnings
warnings.filterwarnings("ignore")

Так как в этом домашнем задании использовать аугментации для обучения __обязательно__ - советуем воспользоваться библиотекой albumentations.

Она  особенно удобна, поскольку умеет сама вычислять новые координаты bounding box'ов после трансформаций картинки. Для знакомства с этим механизмом советуем следующий [гайд](hts://albumentations.ai/docs/getting_started/bounding_boxes_augmentation/).

Вы все еще можете избрать путь torchvision.transforms, вам потребуется знакомый нам метод `__getitem__`, однако вычислять новые координаты bounding box'ов после трансформаций вам придётся вручную

__Обратите внимание__ на то, что в статье коробки предсказаний параметризуются через: _(x_center, y_center, width, height)_ (причем эти значения _относительные_), а в наших файлах - это _(x_min, y_min, x_max, y_max)_

Также, помните что модель должна предсказывать как прямоугольник с обьектом, так и вероятности каждого класса!

In [2]:
def biection(data_dir):
    """"
    Сопоставляет каждой картинке и таблице в папке номер
    -------------------------
    return: словарь с ключами - названием картинки/таблицы, объектами - номерами
    """
    w_path = os.path.join(os.getcwd(),data_dir)
    answ = {}
    count = 0
    for filename in os.listdir(w_path):
        name =  filename.split(".")[0]
        if name not in answ.keys():
            answ[name] = count
            count += 1
    return answ

2 * len(biection("./data/test")) == len(os.listdir(os.path.join(os.getcwd(),"./data/test")))

True

In [3]:
def get_paths(names_to_num, data_dir):
    """"
    Возвращает словарь путей до картинок и таблиц с коробками, причём
    индексация в различных словарях совпадает
    -------------------------
    return: словарь с ключами - цифрами, объектами - ссылки
    """
    w_path = os.path.join(os.getcwd(),data_dir)
    i_path = {}
    b_path = {}
    for name in list(names_to_num.keys()):
        num = names_to_num[name]
        i_path[num] = os.path.join(w_path,(name + ".jpg"))
        b_path[num] = os.path.join(w_path,(name + ".xml"))
    return i_path, b_path

In [4]:
path_1 = '/Users/fedor/DataspellProjects/IDA_HW_3/data/test/orange_77.jpg'
h = np.array(Image.open(path_1).convert("RGB")).shape[0]
w = np.array(Image.open(path_1).convert("RGB")).shape[1]

In [5]:
class2tag = {"apple": 1, "orange": 2, "banana": 3}

class FruitDataset(Dataset):
    def __init__(self, data_dir, transforms=None):
        self.biection = biection(data_dir)
        self.image_paths , self.box_paths = get_paths(self.biection, data_dir)
        assert len(self.image_paths) == len(self.box_paths)
        self.transforms = transforms


    def __getitem__(self, idx):
        image = np.array( Image.open(self.image_paths[idx]).convert("RGB") )
        bboxes, class_labels = self.__get_boxes_from_xml(self.box_paths[idx])
        bboxes = (
            self.__convert_to_yolo_box_params(
                                        bboxes,
                                        image.shape[1],
                                        image.shape[0])
                                                )

        if self.transforms:
            transformed = self.transforms(image = image, bboxes = bboxes, class_labels = class_labels)
            bboxes = transformed['bboxes']
            image = transformed['image']
            class_labels =  transformed['class_labels']

        return torch.Tensor(image).permute(2, 0, 1), (bboxes, class_labels)

    def __len__(self):
        return len(self.image_paths)

    def __get_boxes_from_xml(self, xml_filename: str):
        """
          Метод, который считает и распарсит (с помощью xmltodict) переданный xml
          файл и вернет координаты прямоугольников объектов на соответсвующей фотографии
          и название класса обьекта в каждом прямоугольнике
        --------------------------
        return:
        boxes - format lists of lists [xmin, ymin, xmax, ymax]
        name - list of
        """
        root_node = ET.parse(xml_filename).getroot()
        boxes = []
        class_labels = []

        for object in root_node.iter('object'):# Get the value from the attribute 'object'
            name = object.find('name').text
            xmin = int(object.find('bndbox').find('xmin').text)
            ymin = int(object.find('bndbox').find('ymin').text)
            xmax=  int(object.find('bndbox').find('xmax').text)
            ymax = int(object.find('bndbox').find('ymax').text)

            boxes.append([xmin, ymin, xmax, ymax])
            class_labels.append(class2tag[name])

        return boxes, class_labels

    def __convert_to_yolo_box_params(self, all_box_coordinates: List[int], im_w, im_h):
        """
        Перейти от [xmin, ymin, xmax, ymax] к [x_center, y_center, width, height].

        Обратите внимание, что параметры [x_center, y_center, width, height] - это
        относительные значение в отрезке [0, 1]

        :param: box_coordinates - координаты коробки в формате [xmin, ymin, xmax, ymax]
        :param: im_w - ширина исходного изображения
        :param: im_h - высота исходного изображения

        :return: координаты коробки в формате [x_center, y_center, width, height]
        """
        totall_anw = []
        for box_coordinates in all_box_coordinates:
            ans = []
            ans.append((box_coordinates[0] + box_coordinates[2]) / 2 / im_w)  # x_center
            ans.append((box_coordinates[1] + box_coordinates[3]) / 2 / im_h)  # y_center

            ans.append((box_coordinates[2] - box_coordinates[0]) / im_w)  # width
            ans.append((box_coordinates[3] - box_coordinates[1]) / im_h)  # height
            totall_anw.append(ans)

        return totall_anw

In [6]:
WIDTH, HEIGHT = 448, 448

train_transform = A.Compose([
    albumentations.augmentations.geometric.resize.Resize(height=HEIGHT, width = WIDTH),
    A.RandomRotate90(p=0.3),
    A.HorizontalFlip(p=0.3),
    A.Transpose(p=0.3),
    A.PixelDropout(p=0.5),
    A.Blur(blur_limit=(3,4), p=0.5)
                            ],
                             bbox_params=A.BboxParams(format='yolo',
                                                      label_fields=['class_labels']))
test_transform = A.Compose([
    albumentations.augmentations.geometric.resize.Resize(height=HEIGHT, width = WIDTH)
                            ],
                            bbox_params=A.BboxParams(format='yolo',
                                                     label_fields=['class_labels']))

In [7]:
train_dataset = FruitDataset(
    transforms=train_transform,
    data_dir="./data/train"
    )

val_dataset = FruitDataset(
    transforms=test_transform, 
    data_dir="./data/test"
    )

# Немного проверок, чтобы убедиться в правильности направления решения
assert isinstance(train_dataset[0], tuple)
assert len(train_dataset[0]) == 2
assert isinstance(train_dataset[0][0], torch.Tensor)
assert isinstance(train_dataset[0][1], tuple)
assert len(train_dataset[0][1]) == 2
print("Тесты успешно пройдены")

Тесты успешно пройдены


In [8]:
train_dataloader = DataLoader(
    dataset=train_dataset,
    batch_size= 1,
    shuffle=True)

val_dataloader = DataLoader(
    dataset=val_dataset,
    batch_size=1,
    shuffle=False
    )

Теперь определим функцию для рассчета Intersection Over Union по 4 углам двух прямоугольников

In [9]:
def intersection_over_union(predicted_bbox, gt_bbox) -> float:
    """
    Intersection Over Union для двух прямоугольников

    :param: dt_bbox - [x_min, y_min, x_max, y_max]
    :param: gt_bbox - [x_min, y_min, x_max, y_max]

    :return: Intersection Over Union
    """

    # intersection_bbox = np.array(
    intersection_bbox = torch.Tensor(
        [
            max(predicted_bbox[0], gt_bbox[0]),
            max(predicted_bbox[1], gt_bbox[1]),
            min(predicted_bbox[2], gt_bbox[2]),
            min(predicted_bbox[3], gt_bbox[3]),
        ]
    )

    intersection_area = max(intersection_bbox[2] - intersection_bbox[0], 0) \
                        * max( intersection_bbox[3] - intersection_bbox[1], 0)
    area_dt = (predicted_bbox[2] - predicted_bbox[0]) * (predicted_bbox[3] - predicted_bbox[1])
    area_gt = (gt_bbox[2] - gt_bbox[0]) * (gt_bbox[3] - gt_bbox[1])

    union_area = area_dt + area_gt - intersection_area

    iou = intersection_area / union_area
    return float(iou)

Теперь начинается основная часть домашнего задания: обучите модель YOLO для object detection на __обучающем__ датасете. 

 - Создайте модель и функцию ошибки YoloV1 прочитав [оригинальную статью](https://paperswithcode.com/paper/you-only-look-once-unified-real-time-object)
 - Напишите функцию обучения модели
 - Используйте аугментации

## Реализуйте Модель - 2 балла

Копировать точное количество слоев и параметры сверток необязательно. Главное - чтобы модель работала по принципу, описанному в статье и делала предсказание в представленном формате.


В качестве подсказки напомним, что выходом модели __для каждого обьекта__ должен быть тензор размера
__S * S * (B * 5 + С)__, где все параметры имеют такое же значение, как и в статье: 

- S - количество ячеек на которое разбивается изображение по вертикали/горизонтали
- В - количество предсказываемых прямоугольников в каждой ячейке
- 5 - количество параметров для определения каждого прямоугольника (x_center, y_center, width, height, confidence)
- С - количество классов (apple, banana, orange)

Таким образом, мы для каждого окна размера __S x S__ предсказываем __В__ коробо и один класс

In [10]:
class CNNBlock(nn.Module):  # можно поменять на Lightning
    def __init__(self, convs,out_channels, is_max_pool:bool=False):

        super().__init__()
        self.conv_layers = convs
        self.batchnorm = nn.BatchNorm2d(out_channels)
        self.leakyrelu = nn.LeakyReLU(0.1)

        self.is_maxpool = is_max_pool  # не после каждой свертки нужно делать maxpool
        self.maxpool = nn.MaxPool2d(2,stride=2)

    def forward(self, x):
        x = self.conv_layers(x)
        if len(x.size()) == 3:
            x = x.unsqueeze(0)
        x = self.leakyrelu(self.batchnorm(x))

        if self.is_maxpool:
            x = self.maxpool(x)
        return x

In [11]:
# conv_layer_1 = nn.Sequential(
#     nn.Conv2d(in_channels=3,  out_channels = 192, kernel_size=3, stride =2, padding=1))
#
# conv_layer_2 = nn.Sequential(
#     nn.Conv2d(in_channels=192, out_channels=256, kernel_size=3, padding=1))
#
# conv_layer_3 = nn.Sequential(
#     nn.Conv2d(in_channels=256, out_channels=128, kernel_size=1),
#     nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, padding=1),
#     nn.Conv2d(in_channels=256, out_channels=256, kernel_size=1),
#     nn.Conv2d(in_channels=256, out_channels=512, kernel_size=3, padding=1)
# )
#
# conv_layer_4 = nn.Sequential(
#     nn.Conv2d(in_channels=512, out_channels=256, kernel_size=1),#1
#     nn.Conv2d(in_channels=256, out_channels=512, kernel_size=3, padding=1),
#     nn.Conv2d(in_channels=512, out_channels=256, kernel_size=1),#2
#     nn.Conv2d(in_channels=256, out_channels=512, kernel_size=3, padding=1),
#     nn.Conv2d(in_channels=512, out_channels=256, kernel_size=1),#3
#     nn.Conv2d(in_channels=256, out_channels=512, kernel_size=3, padding=1),
#     nn.Conv2d(in_channels=512, out_channels=256, kernel_size=1),#4
#     nn.Conv2d(in_channels=256, out_channels=512, kernel_size=3, padding=1),
#
#     nn.Conv2d(in_channels=512, out_channels=512, kernel_size=1),
#     nn.Conv2d(in_channels=512, out_channels=1024, kernel_size=3, padding=1)
# )
#
# conv_layer_5 = nn.Sequential(
#     nn.Conv2d(in_channels=1024, out_channels=512, kernel_size=1),#1
#     nn.Conv2d(in_channels=512, out_channels=1024, kernel_size=3, padding=1),
#     nn.Conv2d(in_channels=1024, out_channels=512, kernel_size=1),#2
#     nn.Conv2d(in_channels=512, out_channels=1024, kernel_size=3, padding=1),
#
#     nn.Conv2d(in_channels=1024, out_channels=1024, kernel_size=3, padding=1),
#     nn.Conv2d(in_channels=1024, out_channels=1024, kernel_size=3,stride = 2, padding=1)
# )
#
# conv_layer_6 = nn.Sequential(
#     nn.Conv2d(in_channels=1024, out_channels=1024, kernel_size=3, padding=1),
#     nn.Conv2d(in_channels=1024, out_channels=1024, kernel_size=3, padding=1)
# )

In [12]:
conv_layer_1 = nn.Sequential(
    nn.Conv2d(in_channels=3,  out_channels = 192, kernel_size=3, stride =2, padding=1))

conv_layer_2 = nn.Sequential(
    nn.Conv2d(in_channels=192, out_channels=256, kernel_size=3, padding=1))

conv_layer_3 = nn.Sequential(
    # nn.Conv2d(in_channels=256, out_channels=128, kernel_size=1),
    # nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, padding=1),
    nn.Conv2d(in_channels=256, out_channels=256, kernel_size=1),
    nn.Conv2d(in_channels=256, out_channels=512, kernel_size=3, padding=1)
)

conv_layer_4 = nn.Sequential(
    nn.Conv2d(in_channels=512, out_channels=256, kernel_size=1),#1
    nn.Conv2d(in_channels=256, out_channels=512, kernel_size=3, padding=1),
    nn.Conv2d(in_channels=512, out_channels=256, kernel_size=1),#2
    nn.Conv2d(in_channels=256, out_channels=512, kernel_size=3, padding=1),
    # nn.Conv2d(in_channels=512, out_channels=256, kernel_size=1),#3
    # nn.Conv2d(in_channels=256, out_channels=512, kernel_size=3, padding=1),
    # nn.Conv2d(in_channels=512, out_channels=256, kernel_size=1),#4
    # nn.Conv2d(in_channels=256, out_channels=512, kernel_size=3, padding=1),

    nn.Conv2d(in_channels=512, out_channels=512, kernel_size=1),
    nn.Conv2d(in_channels=512, out_channels=1024, kernel_size=3, padding=1)
)

conv_layer_5 = nn.Sequential(
    nn.Conv2d(in_channels=1024, out_channels=512, kernel_size=1),#1
    nn.Conv2d(in_channels=512, out_channels=1024, kernel_size=3, padding=1),
    # nn.Conv2d(in_channels=1024, out_channels=512, kernel_size=1),#2
    # nn.Conv2d(in_channels=512, out_channels=1024, kernel_size=3, padding=1),

    # nn.Conv2d(in_channels=1024, out_channels=1024, kernel_size=3, padding=1),
    nn.Conv2d(in_channels=1024, out_channels=1024, kernel_size=3,stride = 2, padding=1)
)

conv_layer_6 = nn.Sequential(
    # nn.Conv2d(in_channels=1024, out_channels=1024, kernel_size=3, padding=1),
    nn.Conv2d(in_channels=1024, out_channels=512, kernel_size=3, padding=1)
)

In [13]:
# tests
a  = CNNBlock(conv_layer_1, 192, True)
x = a.forward(train_dataset.__getitem__(idx = 0)[0])
print(x.size() == torch.Size([1, 192, 112, 112]))
b = CNNBlock(conv_layer_2, 256, True)
x = b.forward(x)
print(x.size() == torch.Size([1, 256, 56, 56]))
c = CNNBlock(conv_layer_3, 512, True)
x = c.forward(x)
print(x.size() == torch.Size([1, 512, 28, 28]))
d = CNNBlock(conv_layer_4, 1024, True)
x = d.forward(x)
print(x.size() == torch.Size([1, 1024, 14, 14]))
e = CNNBlock(conv_layer_5, 1024, False)
x = e.forward(x)
print(x.size() == torch.Size([1, 1024, 7, 7]))

f = CNNBlock(conv_layer_6, 512, False) #1024
x = f.forward(x)

print(x.size() == torch.Size([1, 1024, 7, 7]))
l1 = nn.Linear(7*7*512,4096)
out = nn.Flatten()(x)
out = l1(out)
out = nn.LeakyReLU(0.1)(out)
S=7
B=2
C=3
l2 = nn.Linear(4096,S*S*(B*5 + C))
final = l2(out).view(S,S,B*5 + C)
print(final)

True
True
True
True
True
False
tensor([[[-8.9236e-02,  3.7567e-01,  6.2909e-02, -3.0973e-01, -9.4400e-02,
           1.5599e-02,  2.3924e-02,  5.7563e-02,  2.8630e-01,  6.9651e-02,
           7.9393e-02, -1.2314e-01, -8.1455e-02],
         [ 2.1940e-01,  1.8086e-01, -3.3215e-01, -2.6038e-01,  1.3904e-01,
          -4.6683e-02,  1.5579e-02, -5.3946e-02,  6.8055e-02,  3.4724e-01,
          -8.0993e-02,  7.2766e-02, -1.9864e-01],
         [ 5.9243e-03, -2.2191e-03,  5.0802e-02,  2.8722e-01,  2.8957e-01,
           1.4581e-01, -4.0938e-02, -6.8013e-02, -1.4862e-02,  2.5626e-01,
           2.8764e-01,  9.6467e-03,  3.2625e-02],
         [ 6.5186e-02,  2.7643e-01, -1.8231e-01, -8.7535e-03, -1.4182e-01,
           5.1804e-02, -4.9984e-02, -6.1981e-02,  4.7091e-01, -3.7214e-01,
           1.7597e-01, -9.9572e-02, -2.3385e-01],
         [-1.8345e-01, -1.8877e-01, -5.1851e-01,  8.0753e-02,  9.6471e-03,
          -1.4020e-01,  2.6317e-01,  1.7989e-01, -5.8542e-02,  3.7141e-01,
          -2.8631e-

In [14]:
class YOLO(nn.Module):
    def __init__(self, S=7, B=2, C=3):
        """
        :param: S * S - количество ячеек на которые разбивается изображение
        :param: B - количество предсказанных прямоугольников в каждой ячейке
        :param: C - количество классов
        """
        super(YOLO, self).__init__()
        self.S = S
        self.B = B
        self.C = C
        self.conv_block_1 = CNNBlock(conv_layer_1, 192, True)
        self.conv_block_2 = CNNBlock(conv_layer_2, 256, True)
        self.conv_block_3 = CNNBlock(conv_layer_3, 512, True)
        self.conv_block_4 = CNNBlock(conv_layer_4, 1024, True)
        self.conv_block_5 = CNNBlock(conv_layer_5, 1024, False)
        self.conv_block_6 = CNNBlock(conv_layer_6, 512, False) #1024
        self.linear_1 =  nn.Linear(7*7*512,1024) # 7 7 1024, 4096
        self.leaky_relu = nn.LeakyReLU(0.1)
        self.linear_2 =  nn.Linear(1024,S*S*(B*5 + C)) #2048,S*S*(B*5 + C)

    def forward(self, x):
        x = self.conv_block_1(x)
        x = self.conv_block_2(x)
        x = self.conv_block_3(x)
        x = self.conv_block_4(x)
        x = self.conv_block_5(x)
        x = self.conv_block_6(x)
        x = self.leaky_relu(
            self.linear_1(
                nn.Flatten()(x)
            )
        )
        x = self.linear_2(x)
        x= x.view(self.S,self.S,self.B*5 + self.C)
        return x

In [15]:
temp_model = YOLO()
expected_output_shape = temp_model.S * temp_model.S * (5 * temp_model.B + temp_model.C)

assert temp_model(train_dataset[0][0]).reshape(-1).shape[0] == expected_output_shape

In [16]:
# model_yolo = YOLO()
# pred = model_yolo.forward(train_dataset.__getitem__(idx=1)[0])
# pred

## Реализуйте YoloLoss - 3 балла

In [25]:
# S × S × (B ∗ 5 + C) tensor     x, y, w, h, and confidence
# “responsible” for predicting an object based on which
# prediction has the highest current IOU with the ground
# truth
# x, y, w, h, c,    x, y, w, h, c,   cl, cl, cl ---- pred
# x, y, w, h,    cl, cl, cl --- target
class YoloLoss(nn.Module):
    def __init__(self, S=7, B=2, C=3):
        """
        :param: S * S - количество ячеек на которые разбивается изображение
        :param: B - количество предсказанных прямоугольников в каждой ячейке
        :param: C - количество классов
        :param: lambda_noobj - константа для обозначения важности ячеек без объектов
        :param: objects_dict - константа для обозначения важности ячеек с объектами
        :param: objects_dict - словарь
         с ключами -  координатами (i,j) ячеек содержащих объекты
         с значениями - таргет значениями для этих ячеек в формате [(b_box, class)]
        :param: class2vec - словарь
         с ключами числовыми метками классов
         с значениями one-hot векторами этого класса
        """

        super().__init__()
        self.mse = nn.MSELoss(reduction="sum")

        self.S = S
        self.B = B
        self.C = C

        self.lambda_noobj = 0.5
        self.lambda_coord = 5
        self.objects_dict = {}
        self.class2vec = {1:[1,0,0],2:[0,1,0],3:[0,0,1]}

    def _get_best_prediction(self, predictions,target):
        """
        для всех ячеек в матрице S*S находит "ответственные" значения среди В вариантов
        :param predictions: - предсказания модели YOLO
        :param target:  - целевые значения
        :return best_box_mask: torch.Tensor([S,S,B*5+C])best_box_mask - бинарная маска
        """
        predictions = predictions.squeeze(0)
        best_box_mask = torch.zeros([self.S,self.S,self.B*5+self.C])

        for i in range(self.S):
            for j in range(self.S):
                best_box_mask[i,j,-self.C:] = 1
                if (i,j) not in self.objects_dict.keys():
                    best_box_mask[i,j,[4,9]] = 1 # понадобиться когда мы будем уменьшать уверенность для ячеек бкз объекта
                    continue
                max_iou=0
                best_1, best_2 = (0,5)
                for k in range(0,5*self.B,5):

                    if intersection_over_union(predictions[i,j,k:k+4], self.objects_dict[(i,j)][0][0]) >= max_iou:
                        best_1, best_2 = (k,k+5)
                        max_iou = intersection_over_union(predictions[i,j,k:k+4], self.objects_dict[(i,j)][0][0])

                best_box_mask[i,j,best_1: best_2] = 1

        return best_box_mask

    def _dict_init(self, target):
        """
        инициализирует словарь objects_dict при помощи значений из target
        :param target: - таргет значение
        :return: nan
        """
        self.objects_dict = {}
        b_target = target[:-1][0]
        clfsses = target[-1]
        for b_box, clas in zip(b_target, clfsses):
            c  = self._get_cel_cord(b_box)
            if c not in self.objects_dict.keys():
                self.objects_dict[c] = [(b_box,clas)]
            else:
                self.objects_dict[c] = self.objects_dict[c].append((b_box,clas))
        return

    def _get_cel_cord(self, bbox):
        """
        по bounding box возвращает номер ячейки куда приходиться центр
        :param bbox: окаймляющая рамка (bounding box) целевого значения
        :return x, y: int, int - координаты ячейки (её номера)
        """
        count = 0
        x = self.S - 1
        y = self.S-1
        for cord in np.linspace(0,1,self.S+1)[:-1]:
            if cord > bbox[0]:
                x = count
                break
            count += 1

        count = 0
        for cord in np.linspace(0,1,self.S+1)[:-1]:
            if cord > bbox[1]:
                y = count
                break
            count += 1
        return x, y

    def _get_masks(self,target):
        """
         Возращает бинарные маски, указывающие на то, есть ли объект в ячейке.
         :param target: целевые значения
         :return mask_obj: torch.Tensor([S,S,B*5+C]) - маска присутствия центра объекта
         :return mask_noobj: torch.Tensor([S,S,B*5+C]) - маска отсутствия центра объекта
        """
        mask_obj = torch.zeros([self.S,self.S,self.B*5+self.C])
        mask_noobj = torch.ones([self.S,self.S,self.B*5+self.C])
        for i,j in self.objects_dict.keys():
            mask_obj[i,j] = 1
            mask_noobj[i,j] = 0

        return mask_obj,mask_noobj

    def _set_target(self):
        """
        возвращает target значение в формате тензора [S,S,5+C]
        :return new_target:  - torch.Tensor([S,S,5+C]) - target тензор в формате
        """
        new_target = torch.zeros([self.S,self.S,4+self.C])
        for i,j in self.objects_dict.keys():
            l = list(self.objects_dict[(i,j)][0][0])
            self._get_YOLO_coordinates(i,j,l)
            vec = self.class2vec[self.objects_dict[(i,j)][0][1][0].item()]
            l = l+vec
            new_target[i,j,:] = torch.Tensor(l)

        return new_target

    def _get_YOLO_coordinates(self,i,j,b_box):
        """
        :param i: int - first coordinate of the cell
        :param j: int - second coordinate of the cell
        :param b_box: ittarable - bounding box parameters
        :return: bounding box parameters in YOLO format
        """
        b_box[0] = b_box[0] - (i*(1/(self.S)) + (1/(self.S))/2) #relative to center of cell i,j
        b_box[1] = b_box[1] - (j*(1/(self.S)) + (1/(self.S))/2)
        return b_box

    def forward(self, predictions, target):
        """
        Считает loss значение для входных предсказаний и целевых значений
        :param predictions: torch.Tensor([S,S,B*5+C]) - предсказания
        :param target: (b_boxes, classes) - целевые значения
        :return answ: torch.Tensor([1]) - значение loss
        """
        self._dict_init(target)
        mask_obj, mask_noobj = self._get_masks(target)
        predictions = predictions.reshape(-1, self.S, self.S,  self.B * 5 + self.C)
        target = self._set_target()
        best_box = self._get_best_prediction(predictions, target)

        # print(mask_obj)
        # print(mask_noobj)
        # print(predictions)
        # print("target",target)
        # print("best_box", best_box)
        # print(mask_obj*best_box*predictions)
        # X
        answ = self.lambda_coord * self.mse((mask_obj*best_box*predictions)[...,[0]], target[...,0]) + \
               self.lambda_coord * self.mse((mask_obj*best_box*predictions)[...,[5]], target[...,0])

        # print("x",answ)
        # Y
        answ += self.lambda_coord * self.mse((mask_obj*best_box*predictions)[...,[1]], target[...,1]) + \
               self.lambda_coord * self.mse((mask_obj*best_box*predictions)[...,[6]], target[...,1])

        # print("y",answ)
        # W
        answ += self.lambda_coord * self.mse(
            torch.sign((mask_obj*best_box*predictions)[...,[2]]) * torch.sqrt(
                torch.abs((mask_obj*best_box*predictions)[...,[2]])+1e-03),
            torch.sqrt(target[...,2]))+\
            self.lambda_coord * self.mse(
            torch.sign((mask_obj*best_box*predictions)[...,[7]]) *torch.sqrt(
                torch.abs((mask_obj*best_box*predictions)[...,[7]])+1e-03),
            torch.sqrt(target[...,2]))

        # print("w",answ)
        # H
        answ += self.lambda_coord * self.mse(
            torch.sign((mask_obj*best_box*predictions)[...,[3]]) * torch.sqrt(
                torch.abs((mask_obj*best_box*predictions)[...,[3]])+1e-03),
            torch.sqrt( target[...,3]) )+\
            self.lambda_coord * self.mse(
            torch.sign((mask_obj*best_box*predictions)[...,[8]]) *torch.sqrt(
                torch.abs((mask_obj*best_box*predictions)[...,[8]])+1e-03),
            torch.sqrt( target[...,3]))

        # print("h",answ)
        # C mask_obj
        answ += self.mse((mask_obj*best_box*predictions)[...,[4]], best_box[...,[4]]*mask_obj[...,[4]]*torch.ones([self.S,self.S])) + \
               self.mse((mask_obj*best_box*predictions)[...,[9]], best_box[...,[4]]*mask_obj[...,[4]]*torch.ones([self.S,self.S]))

        # print("c_obj",answ)
        # C mask_noobj
        answ += self.lambda_noobj * self.mse((mask_noobj*best_box*predictions)[...,[4]], torch.zeros([self.S,self.S])) + \
               self.lambda_noobj * self.mse((mask_noobj*best_box*predictions)[...,[9]], torch.zeros([self.S,self.S]))

        # print("c__nooj",answ)
        # Classes
        answ += self.mse((mask_obj*predictions)[...,[10,11,12]],
                                                target[...,[4,5,6]])
        return answ

## Обучение...
Я знаю, что не успел нормально дообучить! Времени не хватило от слова совсем... Но по тем лоссам которые приведены у меня, видно что лосс-то падает! Я знаю, что на меня разгневаються боги ML-я, за то что я не построил граффики, но построить их правда не успел.


In [339]:
y_pred = torch.Tensor()
f_loss = YoloLoss()
y = torch.Tensor()
for x, y in train_dataloader:
    y_pred = model_yolo(x)
    y = y
    break

In [340]:

f_loss = YoloLoss()
loss = f_loss(y_pred,y)
loss

tensor(133.7650, grad_fn=<AddBackward0>)

In [197]:
class Fake_Net(nn.Module):  # можно поменять на Lightning
    def __init__(self):
        super().__init__()
        self.conv_layers = nn.Conv2d(in_channels=3,  out_channels = 3, kernel_size=3, stride =2)
    def forward(self, x):
        x = self.conv_layers(x)
        x = nn.Flatten()(x)
        return x

In [26]:
model_yolo = YOLO()
f_loss = YoloLoss()
optimizer = torch.optim.Adam( model_yolo.parameters(),lr=0.001,eps=1e-08)

# fake_loss = nn.MSELoss(reduction="sum")
# fake_net = Fake_Net()
# fake_optim = torch.optim.Adam(fake_net.parameters(), lr=1e-3, eps=1e-8 )

In [29]:
# та часть где я не понимаю что происходит
for x, y in train_dataloader:
    # model.train()
    y_pred = model_yolo(x)

    # y_pred = fake_net(x)
    # y_pred = nn.Flatten()(y_pred)
    # loss = fake_loss(y_pred,torch.zeros(y_pred.size()))

    loss = f_loss(y_pred,y)
    loss.backward()

    # print("y_pred ",y_pred)
    optimizer.step()
    optimizer.zero_grad()

    print("loss ",loss)

loss  tensor(4237.5059, grad_fn=<AddBackward0>)
loss  tensor(11473.1641, grad_fn=<AddBackward0>)
loss  tensor(1030.8716, grad_fn=<AddBackward0>)
loss  tensor(3069.4968, grad_fn=<AddBackward0>)
loss  tensor(1778.0552, grad_fn=<AddBackward0>)
loss  tensor(5566.4761, grad_fn=<AddBackward0>)
loss  tensor(12841.4365, grad_fn=<AddBackward0>)
loss  tensor(1623.9628, grad_fn=<AddBackward0>)
loss  tensor(7613.2349, grad_fn=<AddBackward0>)
loss  tensor(1895.7800, grad_fn=<AddBackward0>)
loss  tensor(3618.0898, grad_fn=<AddBackward0>)
loss  tensor(1085.3815, grad_fn=<AddBackward0>)
loss  tensor(3683.2000, grad_fn=<AddBackward0>)
loss  tensor(14529.6621, grad_fn=<AddBackward0>)
loss  tensor(24587.8438, grad_fn=<AddBackward0>)
loss  tensor(14471.3115, grad_fn=<AddBackward0>)
loss  tensor(3550.0630, grad_fn=<AddBackward0>)
loss  tensor(1174.7939, grad_fn=<AddBackward0>)
loss  tensor(1400.7972, grad_fn=<AddBackward0>)
loss  tensor(1116.2472, grad_fn=<AddBackward0>)
loss  tensor(911.4396, grad_fn=<Add

KeyboardInterrupt: 

## Реализуйте дополнительные функции из статьи - 2 балла

In [ ]:
def non_max_suppression(bboxes, iou_threshold, threshold):
    """
    Non max suppression for list of hypotheses

    :bboxes: List[torch.tensor] list of coordinates of bounding boxes and there conference
    :iou_threshold: float threshold for intersection_over_union function

    :returns: List[torch.tensor] list of coordinates of bounding boxes
    """
    bboxes = bboxes.sort(lambda b_box: b_box[4])
    answ = [bboxes[0]]
    for bbox in bboxes[1:]:
        for answ_box in answ:
            if intersection_over_union(bbox, answ_box) > iou_threshold and bbox[4] >= threshold:
                answ.append(bbox)
    return answ

def mean_average_precision(pred_boxes, true_boxes, iou_threshold=0.5):
    pass

def get_bound_boxes(loader, model, iou_threshold=0.5, threshold=0.4):
    ## YOUR CODE
    return all_pred_boxes, all_true_boxes

## Обучите модель и посчитайте метрики для задачи детекции - 2 балла 

Несмотря на то, что в этом блоке ничего сильно нового для вас не ожидается и за него формально дается лишь два балла - провести обучение очень важно для понимания того, насколько правильно реализована ваша модель и лосс.

В процессе обучения будет видно все ли размерности совпадают, падает ли лосс и растут ли метрики целевой задачи, поэтому на практике этот пункт гораздо оказывается гораздо важнее.

In [ ]:
from datetime import date, datetime
version = str(datetime.now().strftime("%H:%M(%d.%m)"))
import wandb
wandb.login()


version = str(datetime.now().strftime("%H:%M(%d.%m)"))
run = wandb.init(project="iad_hw_2", name=f'resnet18 version {version}')

In [22]:
class YOLOLearner(pl.LightningModule):
    def __init__(self, model, loss) -> None:
        super().__init__()
        self.model = model
        self.loss = loss
        self.optimizer = torch.optim.Adam(self.model.parameters(), lr=1e-3)

    def forward(self, x) -> torch.Tensor:
        return self.model(x)

    def configure_optimizers(self):
        return self.optimizer

    def training_step(self, train_batch, batch_idx) -> torch.Tensor:
        x,y = train_batch

        y_pred = model_yolo(x)
        loss = f_loss.forward(y_pred,y)
        print("y_pred ",y_pred)

        return loss

In [23]:
model = YOLO()
loss_function = YoloLoss()
n_epochs = 10

yolo_learner = YOLOLearner(model,loss_function )  ## YOUR CODE

device = "gpu" if torch.cuda.is_available() else "cpu"
trainer = pl.Trainer(accelerator=device, max_epochs=n_epochs)

trainer.fit(yolo_learner, train_dataloader, val_dataloader)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Missing logger folder: /Users/fedor/DataspellProjects/IDA_HW_3/lightning_logs

  | Name  | Type     | Params
-----------------------------------
0 | model | YOLO     | 55.1 M
1 | loss  | YoloLoss | 0     
-----------------------------------
55.1 M    Trainable params
0         Non-trainable params
55.1 M    Total params
220.208   Total estimated model params size (MB)


Training: 0it [00:00, ?it/s]

TypeError: unsupported operand type(s) for *: 'int' and 'CNNBlock'

## Посчитайте метрики задачи детекции на валидационной выборке

Попробуйте понять насколько хороши ваши показатели. Если числа кажутся подозрительно низкими - возможно вам стоит перепроверить ваше решение.

In [ ]:
## YOUR CODE

## Визуализируйте предсказанные bounding box'ы для любых пяти картинок из __валидационного__ датасета.

In [ ]:
image, targets = next(iter(val_dataset))
preds = ## YOUR CODE

In [ ]:
from PIL import ImageDraw

image = torchvision.transform.ToPILImage()(image)
draw = ImageDraw.Draw(image)

for box in targets[0]:
    ## YOUR CODE
    draw.rectangle([(box[0], box[1]), (box[2], box[3])])

for box in preds[0]:
    ## YOUR CODE
    draw.rectangle([(box[0], box[1]), (box[2], box[3])], outline='red')
image